In [ ]:
from IPython.display import HTML, SVG
HTML(open('../style.css').read())

In [ ]:
%load_ext nb_mypy

# Nim

This notebook defines a solver for the game [Nim](https://en.wikipedia.org/wiki/Nim).
The particular instance of Nim that you are going to implement in this notebook works as shown below:
<img src="NimGame.svg"  width="200">

The game works as follows:
 * There are four rows of matches:
   - the first  row contains 1 match,
   - the second row contains 3 matches,
   - the third  row contains 5 matches, and
   - the fourth row contains 7 matches.
 * The player whose turn it is first selects a line.  
   Then he takes any number of matches from this line.
 * The player that takes the last match has won the game. 

`Board` is a type alias for a list of `int` values. `Row` represents the number of a row of the board.

In [ ]:
Board = list[int]
Row   = int
Move  = tuple[Row, int]

`gSize` is a global variable that defines the number of rows.

In [ ]:
gSize = 8

The function `move(b, r, n)` creates a new board by taking `n` matches from
row `r` on the board `b` and returns the resulting board.

```
isWinning :: Int -> Int -> Bool
isWinning s n = 0 <= n ⊕ s && n ⊕ s < n

-- all wining moves for the board `b`
winningMoves :: Board -> [(Int, Int)]
winningMoves b = [ (r, n - n ⊕ s) | (r, n) <- zip [1..] b, isWinning s n ]
  where s = nimSum b
```

In [ ]:
def move(b: Board, r: Row, n: int) -> Board:
    assert n <= b[r], "computer takes more matches than are available"
    return b[:r] + [b[r] - n] + b[r+1:]

Check whether there is a winning move for a row containing `n` matches.
The parameter `s` is the *xor* sum of the current board.

In [ ]:
def is_winning(s: int, n: int) -> bool:
    return 0 <= n ^ s < n

Compute the Nim sum of the board.

In [ ]:
def nim_sum(b: Board) -> int:
    if b == []:
        return 0
    return b[0] ^ nim_sum(b[1:])

Return a list of all winning moves for the board `b`.  A *move* is represented as a pair `(r, n)`
where `n` matches are taken from row `r`.

In [ ]:
def winning_moves(b: Board) -> list[Move]:
    s = nim_sum(b)
    return [(r, n - (n ^ s)) for r, n in enumerate(b) if is_winning(s, n)]

In [ ]:
winning_moves([1, 2, 3, 4, 5, 6, 7, 8])

Given a board `b` with no possible winning moves, this function returns 
the first legal move.

In [ ]:
def any_move(b: Board) -> Move:
    for r, n in enumerate(b):
        if n > 0:
            return (r, 1) 
    return None # type: ignore

Return an optimal move for the given board `b`.

In [ ]:
def best_move(b: Board) -> Move:
    WM = winning_moves(b)
    if WM != []:
        return WM[0]
    return any_move(b)

# Input / Output

In [ ]:
def print_board(b: Board):
    for i, r in enumerate(b):
        print(f'{i}: {"* " * r}')
    print(f"Current Nim sum: {nim_sum(b)} = 0b{nim_sum(b):04b}")

In [ ]:
print_board([1,2,3,4,5,6,7,8])

The following forward declaration is needed by the type checker.

In [ ]:
def play_computer(b: Board) -> None:
    pass

Ask the human player to make a move.

In [ ]:
def play_human(b: Board) -> None:
    print_board(b)
    r = -1 
    while r < 0:
        pair = input("\nEnter your move in the format (row, number): ")
        r, n = map(int, pair.strip("()").split(","))
        if b[r] >= n:
            break
        print(f"Too few matches in row {r}. Try again.")        
    b = move(b, r, n)
    if sum(b) == 0:
        print_board(b)
        print("The human player has won!\n")
    else:
        play_computer(b)

Let the computer make a move on the given board.

In [ ]:
def play_computer(b: Board) -> None:
    print_board(b)
    r, n = best_move(b)
    print(f"The computer takes {n} matches from row {r}.")
    b = move(b, r, n)
    if sum(b) == 0:
        print("The computer has won!\n")
        print_board(b)
    else:
        play_human(b)

Function to start the game.

In [ ]:
def main():
    play_human(list(range(1, gSize + 1)))    

In [ ]:
main()